In [1]:
#导入库
import pandas as pd
import numpy as np
import time
pd.set_option('display.max_columns', 300)

In [2]:
#读取数据
df1 = pd.read_excel('表1-患者列表及临床信息.xlsx')
df2 = pd.read_excel('表2-患者影像信息血肿及水肿的体积及位置.xlsx')
df3 = pd.read_excel('附表1-检索表格-流水号vs时间.xlsx')

In [3]:
#构建数据结构
df1 = df1.rename(columns={'Unnamed: 0':'ID'})
df3 = df3.rename(columns={'入院首次检查流水号':'首次检查流水号'})

In [4]:
df3 = df3.iloc[:, :20]

In [5]:
data = df1.merge(df2, on='ID', how='left')

In [6]:
data = data.query("数据集划分 == '训练'").copy()

In [7]:
data = data[['ID', '发病到首次影像检查时间间隔', '首次检查流水号', 'HM_volume', 
     '随访1流水号', 'HM_volume.1', '随访2流水号', 'HM_volume.2', '随访3流水号', 'HM_volume.3', 
     '随访4流水号', 'HM_volume.4', '随访5流水号', 'HM_volume.5', '随访6流水号', 'HM_volume.6', 
     '随访7流水号', 'HM_volume.7', '随访8流水号', 'HM_volume.8']]

In [8]:
#数据拼接
data = data.merge(df3[['首次检查流水号', '入院首次检查时间点']], on='首次检查流水号', how='left')

In [9]:
data = data.merge(df3[['随访1流水号', '随访1时间点']], on='随访1流水号', how='left')

In [10]:
data = data.merge(df3[['随访2流水号', '随访2时间点']], on='随访2流水号', how='left')

In [11]:
data = data.merge(df3[['随访3流水号', '随访3时间点']], on='随访3流水号', how='left')

In [12]:
data = data.drop_duplicates()

In [13]:
data = data.merge(df3[['随访4流水号', '随访4时间点']], on='随访4流水号', how='left')

In [14]:
data = data.drop_duplicates()

In [15]:
data = data.merge(df3[['随访5流水号', '随访5时间点']], on='随访5流水号', how='left')

In [16]:
data = data.drop_duplicates()

In [17]:
data = data.merge(df3[['随访6流水号', '随访6时间点']], on='随访6流水号', how='left')

In [18]:
data = data.drop_duplicates()

In [19]:
data = data.merge(df3[['随访7流水号', '随访7时间点']], on='随访7流水号', how='left')

In [20]:
data = data.drop_duplicates()

In [21]:
data = data.merge(df3[['随访8流水号', '随访8时间点']], on='随访8流水号', how='left')

In [22]:
data = data.drop_duplicates()

In [23]:
data = data.fillna(0)

In [24]:
data[['随访1流水号','随访2流水号', '随访3流水号', '随访4流水号', '随访5流水号', '随访6流水号', '随访7流水号', '随访8流水号']] = data[['随访1流水号','随访2流水号', '随访3流水号', '随访4流水号', '随访5流水号', '随访6流水号', '随访7流水号', '随访8流水号']].astype(np.int64)

In [25]:
#data.to_csv('result.csv', index=None)

In [26]:
date_dict = {'随访1流水号': ['随访1时间点', 'HM_volume.1'], '随访2流水号': ['随访2时间点', 'HM_volume.2'], 
             '随访3流水号': ['随访3时间点', 'HM_volume.3'], '随访4流水号': ['随访4时间点', 'HM_volume.4'], 
             '随访5流水号': ['随访5时间点', 'HM_volume.5'], '随访6流水号': ['随访6时间点', 'HM_volume.6'], 
            '随访7流水号': ['随访7时间点', 'HM_volume.7'], '随访8流水号': ['随访8时间点', 'HM_volume.8']}

In [27]:
label_dict = {}
time_dict = {}

In [28]:
#判断48小时之内是否发生血肿，并计算发生血肿的时间
for i in data.iterrows():
    i = i[1]
    label_dict[i['ID']] = 0
    time_dict[i['ID']] = 0
    changshu_date = i['发病到首次影像检查时间间隔']
    changshu_vol = i['HM_volume']
    for num in date_dict:
        if i[num] == 0:
            break
        else:
            time1 = int(time.mktime(time.strptime(str(i['入院首次检查时间点']), "%Y-%m-%d %H:%M:%S")))
            time2 = int(time.mktime(time.strptime(str(i[date_dict[num]][0]), "%Y-%m-%d %H:%M:%S")))
            delta = (time2 - time1) / 60 / 60
            time_ = delta + changshu_date
            vol = i[date_dict[num]][1] - changshu_vol
            vol_rate = vol / changshu_vol
            if (time_ <= 48) and ((vol >= 6000) or (vol_rate >= 0.33)):
                label_dict[i['ID']] = 1
                time_dict[i['ID']] = time_
                break

In [31]:
data['label'] = data['ID'].map(label_dict)

In [33]:
data['esp_time'] = data['ID'].map(time_dict)

In [35]:
data.to_csv('fin_result.csv', index=None)

In [36]:
data

,ID,发病到首次影像检查时间间隔,首次检查流水号,HM_volume,随访1流水号,HM_volume.1,随访2流水号,HM_volume.2,随访3流水号,HM_volume.3,随访4流水号,HM_volume.4,随访5流水号,HM_volume.5,随访6流水号,HM_volume.6,随访7流水号,HM_volume.7,随访8流水号,HM_volume.8,入院首次检查时间点,随访1时间点,随访2时间点,随访3时间点,随访4时间点,随访5时间点,随访6时间点,随访7时间点,随访8时间点,label,esp_time
0,sub001,2.5,20161212002136,69714,20161213000009,74902.0,20161218000100,70952.0,20161223001020,62831.0,20161230000574,44029.0,0,0.0,0,0.0,0,0.0,0,0.0,2016-12-12 23:32:54,2016-12-13 05:19:00,2016-12-18 09:09:24,2016-12-23 16:47:09,2016-12-30 14:35:12,0,0,0,0,1,8.268333
160,sub002,3.0,20160406002131,47500,20160407000424,52271.0,20160409000953,47748.0,20160425000901,13055.0,20160523000682,20.0,0,0.0,0,0.0,0,0.0,0,0.0,2016-04-06 21:21:03,2016-04-07 09:16:27,2016-04-09 15:34:22,2016-04-25 10:22:02,2016-05-23 09:32:34,0,0,0,0,1,14.923333
320,sub003,2.0,20160413000006,86396,20160413000428,106042.0,20160414001379,103263.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,2016-04-13 01:18:17,2016-04-13 08:49:38,2016-04-14 14:54:02,0,0,0,0,0,0,1,9.522500
480,sub004,1.0,20161215001667,45498,20161216001119,39877.0,20161219000591,16622.0,20161225000084,8441.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,2016-12-15 22:53:41,2016-12-16 14:52:08,2016-12-19 09:44:24,2016-12-25 09:23:45,0,0,0,0,0,0,0.000000
640,sub005,5.0,20161222000978,14832,20161223000561,24472.0,20161226000628,25477.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,2016-12-22 13:18:48,2016-12-23 10:46:51,2016-12-26 10:15:23,0,0,0,0,0,0,1,26.467500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14882,sub096,1.0,20180802001789,9114,20180803000170,9030.0,20180806000846,8631.0,20180813000513,6573.0,20180820000760,492.0,20180827001569,566.0,0,0.0,0,0.0,0,0.0,2018-08-02 19:28:50,2018-08-03 08:14:31,2018-08-06 10:27:39,2018-08-13 09:35:59,2018-08-20 09:58:54,2018-08-27 14:07:24,0,0,0,0,0.000000
15042,sub097,3.0,20181010000767,10806,20181011001620,12517.0,20181017000365,9929.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,2018-10-10 09:21:10,2018-10-11 14:16:42,2018-10-17 09:45:11,0,0,0,0,0,0,1,7.925556
15202,sub098,2.5,20180612002507,188134,20180614001471,214629.0,20180620002065,178037.0,20180706001349,111107.0,20180720001168,75878.0,0,0.0,0,0.0,0,0.0,0,0.0,2018-06-12 22:32:20,2018-06-14 14:47:44,2018-06-20 15:41:45,2018-07-06 14:32:09,2018-07-20 11:24:26,0,0,0,0,1,18.756667
15362,sub099,3.0,20180620002296,37309,20180621000781,58149.0,20180622000871,52350.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,2018-06-20 19:03:09,2018-06-21 09:43:31,2018-06-22 10:34:55,0,0,0,0,0,0,1,17.672778
